In [76]:
import pandas as pd
from datetime import datetime
import gdown
import sys
import os
import subprocess
import openai
import requests
from dotenv import load_dotenv

load_dotenv()

openai.api_key = os.getenv("OPENAI_API_KEY")

In [1]:
data = {
    'bank': "1dzL_SWBkBs5xrUxuGQTm04oe3USgkL9u",    # banking data
    'sales': "1QK-VgSU3AxXUw330KjYFUj8S9hzKJsG6",   # sales data
    'mcc': "1JN0bR84sgZ_o4wjKPBUmz45NeEEkVgt7",     # mcc description
}

# Download all files from Google Drive
for name, file_id in data.items():
    gdown.download(f'https://drive.google.com/uc?id={file_id}', name + '.parquet', quiet=False)
    
# Read all files and store on a dictionary of pandas dataframes
df = {} 
for name in data.keys():
    df[name] = pd.read_parquet(name + '.parquet')
    

Downloading...
From: https://drive.google.com/uc?id=1dzL_SWBkBs5xrUxuGQTm04oe3USgkL9u
To: /Users/riley/VSCode/HackBRASA/backend/bank.parquet
100%|██████████| 1.57M/1.57M [00:00<00:00, 14.3MB/s]
Downloading...
From: https://drive.google.com/uc?id=1QK-VgSU3AxXUw330KjYFUj8S9hzKJsG6
To: /Users/riley/VSCode/HackBRASA/backend/sales.parquet
100%|██████████| 6.37M/6.37M [00:00<00:00, 16.0MB/s]
Downloading...
From: https://drive.google.com/uc?id=1JN0bR84sgZ_o4wjKPBUmz45NeEEkVgt7
To: /Users/riley/VSCode/HackBRASA/backend/mcc.parquet
100%|██████████| 57.3k/57.3k [00:00<00:00, 1.64MB/s]


In [78]:
# Useful Functions
def display_head_with_scroll(df, num_rows=5):
    """
    Display a DataFrame with horizontal scrolling enabled.
    
    Parameters:
    df (pd.DataFrame): The DataFrame to display.
    num_rows (int): The number of rows to display. Default is 5.
    """
    display(HTML(scrolling_css + df.head(num_rows).to_html(classes='dataframe-div')))

def require(package):
    try:
        # Try to import the package
        globals()[package] = __import__(package)
        return True
    except ImportError:
        print(f"{package} is not installed. Trying to install via Homebrew...")
        try:
            # Attempt to install the package using Homebrew
            subprocess.check_call(["brew", "install", package])
            # After installation, try importing again
            globals()[package] = __import__(package)
            return True
        except subprocess.CalledProcessError:
            print(f"Failed to install {package} via Homebrew. Trying pip with --break-system-packages...")
            try:
                # If Homebrew fails, try installing via pip with the break-system-packages flag
                subprocess.check_call([sys.executable, "-m", "pip", "install", package, "--break-system-packages"])
                globals()[package] = __import__(package)
                return True
            except ImportError:
                print(f"Failed to install {package}.")
                return False
            
import requests

# Replace with your OpenAI API key


# Initialize global variables
newprompt = ""
output = ""
total_tokens_used = 0
cost = 0.0

def openai_prompting(prompt):
    global newprompt
    global output
    global total_tokens_used
    global cost
    print("\n\nRunning GPT-3.5")

    # Define the endpoint URL
    url = "https://api.openai.com/v1/chat/completions"

    # Set up the request headers with your API key
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {openai.api_key}"
    }

    # Define the request payload (input text and parameters)
    data = {
        "model": "gpt-3.5-turbo",  # choose the model
        "messages": [{"role": "system", "content": "You are a data scientist with the goal of analyzing the data in response to the users prompt. The data, stored in df['sales], contains information about sales"
                      " data and has been preprocessed already such that it has df['sales'].shape of (264933, 7), df['sales'].columns.tolist() of ['document_id', 'date_time', 'value', 'card_number', 'type', 'mcc', 'state'],"
                      " and df['sales'].dtypes.tolist() of [dtype('int64'), dtype('<M8[us]'), dtype('float64'), dtype('O'), dtype('O'), dtype('int64'), dtype('O')]. Please make sure to return only python code that is executable"
                      "for querying the data. You can use numpy or pandas to complete the task in response to the users prompt. I have attached the first row of the data as an example. The code should output the answer to the user"
                      " in the simplest manner possible such as an integer with a $."},
                      {"role": "user", "content": f"{prompt}"}],  # prompt here
        "max_tokens": 800  # maximum number of tokens for the model
    }
    if prompt != "":
        newprompt = prompt
        response = requests.post(url, json=data, headers=headers)
    else:
        print("You have an empty prompt, so printing the previous prompt again or default if first prompt is empty.\n")
        print(f"Prompt: {newprompt}")
        print(f"\nOutput: {output}")
        print("\nTokens Used: " + str(total_tokens_used))
        print("Cost: $" + format(cost, ".8f").rstrip("0").rstrip("."))
        return

    # Check if request was successful (status code 200)
    if response.status_code == 200:
        # Parse response to get the text and number of tokens
        output = response.json()['choices'][0]['message']['content']
        output = output.strip().replace("\n\n", "\n")
        prompt_tokens_used = response.json()['usage']['prompt_tokens']
        completion_tokens_used = response.json()['usage']['completion_tokens']
        total_tokens_used = response.json()['usage']['total_tokens']
        
        # Pricing based on gpt-3.5-turbo
        cost_per_input_token = 0.002 / 1_000  # $0.002 per 1,000 tokens for inputs
        cost_per_output_token = 0.002 / 1_000  # $0.002 per 1,000 tokens for outputs
        cost = prompt_tokens_used * cost_per_input_token + completion_tokens_used * cost_per_output_token

        # Print the completion text, tokens used, and cost
        print(f"Prompt: {newprompt}")
        print(f"\nOutput: {output}")
        print("\nTokens Used: " + str(total_tokens_used))
        print("Cost: $" + format(cost, ".8f").rstrip("0").rstrip("."))
        return output
    else:
        # Print error message if request was not successful
        print("Error:", response.text)


In [38]:
# Preprocess the sales data
df['sales']['date_time'] = pd.to_datetime(df['sales']['date_time'])  # Convert to datetime
df['sales']['day_of_week'] = df['sales']['date_time'].dt.day_name()  # Extract day of the week
df['sales']['hour'] = df['sales']['date_time'].dt.hour              # Extract hour of the day

In [28]:
df['sales'].dtypes.tolist()

[dtype('int64'),
 dtype('<M8[us]'),
 dtype('float64'),
 dtype('O'),
 dtype('O'),
 dtype('int64'),
 dtype('O')]

In [13]:
def parse_query(query):
    # Check for specific day queries (e.g., "Friday")
    days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
    for day in days:
        if day.lower() in query.lower():
            return {"type": "day_query", "day": day}
    
    # Add more parsing rules as needed for time ranges, specific dates, etc.
    return None

In [18]:
def query_data(parsed_query):
    if parsed_query["type"] == "day_query":
        day = parsed_query["day"]
        # Sum sales for the specific day of the week
        sales = df['sales'][df['sales']['day_of_week'] == day]['value'].sum()
        return sales

    # Add more query types as needed (e.g., time-based queries)
    return None

In [19]:
user_query = "How were sales on Friday?"

# Step 1: Parse the query
parsed_query = parse_query(user_query)

# Step 2: Query the sales data
result = query_data(parsed_query)

# Step 3: Display the result
if result is not None:
    print(f"Total sales on {parsed_query['day']}: ${result}")
else:
    print("No matching data found.")

Total sales on Friday: $144457633.88


In [79]:
# Define the user query
user_query = "Quais foram essas vendas no fim de semana de janeiro de 2023?"

# Extract the first five rows of df['sales'] as an example to include in the prompt
sample_data = df['sales'].head(1).to_string(index=False)

# Ask the model to generate a prompt that includes the user query and lets it know about the data
generation_prompt = f"""
Given the following user query:

'{user_query}'

Please generate a prompt that includes the necessary instructions and informs the model that it will be expected to analyze the data. The data will be provided, and you can assume that the data is in a Pandas DataFrame. Below are the first five rows of the DataFrame that will be used for analysis:

{sample_data}

Please generate a prompt that explains what the model should do based on the user's query and the data provided.
"""

# Send the request to OpenAI to generate the prompt
response_text = openai_prompting(user_query)

# Extract the generated prompt
print("Generated Prompt:\n\n", output)



Running GPT-3.5
Prompt: Quais foram essas vendas no fim de semana de janeiro de 2023?

Output: ```python
import pandas as pd
# Filtrando as vendas no fim de semana de janeiro de 2023
sales_weekend_jan_2023 = df['sales'][(df['sales']['date_time'].dt.month == 1) & 
                                    (df['sales']['date_time'].dt.year == 2023) & 
                                    (df['sales']['date_time'].dt.dayofweek >= 5)]
# Somando os valores das vendas
total_sales_weekend_jan_2023 = sales_weekend_jan_2023['value'].sum()
total_sales_weekend_jan_2023
```

Tokens Used: 367
Cost: $0.000734
Generated Prompt:

 ```python
import pandas as pd
# Filtrando as vendas no fim de semana de janeiro de 2023
sales_weekend_jan_2023 = df['sales'][(df['sales']['date_time'].dt.month == 1) & 
                                    (df['sales']['date_time'].dt.year == 2023) & 
                                    (df['sales']['date_time'].dt.dayofweek >= 5)]
# Somando os valores das vendas
total_sales_weeke

In [83]:
print(output)

import pandas as pd
# Filtrando as vendas no fim de semana de janeiro de 2023
sales_weekend_jan_2023 = df['sales'][(df['sales']['date_time'].dt.month == 1) & 
                                    (df['sales']['date_time'].dt.year == 2023) & 
                                    (df['sales']['date_time'].dt.dayofweek >= 5)]
# Somando os valores das vendas
total_sales_weekend_jan_2023 = sales_weekend_jan_2023['value'].sum()
total_sales_weekend_jan_2023


In [84]:
import pandas as pd
# Filtrando as vendas no fim de semana de janeiro de 2023
sales_weekend_jan_2023 = df['sales'][(df['sales']['date_time'].dt.month == 1) & 
                                    (df['sales']['date_time'].dt.year == 2023) & 
                                    (df['sales']['date_time'].dt.dayofweek >= 5)]
# Somando os valores das vendas
total_sales_weekend_jan_2023 = sales_weekend_jan_2023['value'].sum()
total_sales_weekend_jan_2023

np.float64(3419941.52)

In [81]:
output = output.replace("```python\n", "").replace("\n```", "")
# Prepare a dictionary to capture the local variables
local_vars = {}

# Execute the code line by line
for line in output.strip().split('\n'):
    exec(line, globals(), local_vars)

# Print the result
formatted_result = "${:,.2f}".format(local_vars['sales_afternoon_2023'])
print(formatted_result)

SyntaxError: '[' was never closed (<string>, line 1)